In [802]:
import pandas as pd
import sqlite3

# Analysis

In [791]:
con = sqlite3.connect('swapi.db')

In [793]:
# LEFT JOIN for male mammal characters, whose homewoeld population is over 1000
query = """
SELECT
    p.name AS character_name,
    pl.name AS homeworld,
    pl.population AS homeworld_population,
    COUNT(f.title) AS film_number,
    s.name AS species_name
FROM
    people AS p
-- add homeworld information
LEFT JOIN
    planets AS pl
    ON p.homeworld_id = pl.id
-- add films, where the character played
LEFT JOIN
    people_to_films AS pf
    ON p.id = pf.id
LEFT JOIN
    films AS f
    ON pf.films_id = f.id
-- add species type
LEFT JOIN
    species_to_people AS sp
    ON p.id = sp.people_id
LEFT JOIN
    species AS s
    ON sp.id = s.id
WHERE
    p.gender LIKE 'male'
    AND pl.population > 1000
    AND s.classification LIKE 'mammal'
GROUP BY
    p.name
ORDER BY
    p.name;

"""

pd.set_option('display.max_colwidth', None)
pd.read_sql_query(sql=query, con=con)

,character_name,homeworld,homeworld_population,film_number,species_name
0,Bail Prestor Organa,Alderaan,2000000000,2,Human
1,Chewbacca,Kashyyyk,45000000,4,Wookie
2,Darth Maul,Dathomir,5200,1,Zabrak
3,Dooku,Serenno,unknown,2,Human
4,Eeth Koth,Iridonia,unknown,2,Zabrak
5,Ki-Adi-Mundi,Cerea,450000000,3,Cerean
6,Nien Nunb,Sullust,18500000000,1,Sullustan
7,San Hill,Muunilinst,5000000000,1,Muun
8,Sebulba,Malastare,2000000000,1,Dug
9,Tarfful,Kashyyyk,45000000,1,Wookie


In [794]:
# INNER JOIN to check if the characters from the films represent the diversity of eye colors in their homeworld

query = """
SELECT
    pl.name AS planet_name,
    pl.population AS planet_population,
    GROUP_CONCAT(DISTINCT ptc.climate) AS planet_climate,
    GROUP_CONCAT(DISTINCT s.name) AS planet_species,
    COUNT(DISTINCT ptyc.eye_color) AS characters_eye_color,
    COUNT(DISTINCT stec.eye_colors) AS species_eye_color,
    COUNT(DISTINCT ptyc.eye_color) * 1.0 /COUNT(DISTINCT stec.eye_colors) AS eye_color_representation,
    GROUP_CONCAT(DISTINCT p.name) AS residents
FROM
    planets AS pl
INNER JOIN
    planets_to_residents AS pr
    ON pl.id = pr.id
INNER JOIN
    people AS p
    ON pr.residents_id = p.id
INNER JOIN
    species AS s
    ON s.homeworld_id = pl.id
INNER JOIN
   planets_to_climate AS ptc
    ON p.id = ptc.id
INNER JOIN
    people_to_eye_color AS ptyc
    ON p.id = ptyc.id
INNER JOIN 
    species_to_eye_colors  AS stec
    ON stec.id = s.id
GROUP BY
    pl.name
ORDER BY
    planet_name;

"""

pd.read_sql_query(sql=query, con=con)



,planet_name,planet_population,planet_climate,planet_species,characters_eye_color,species_eye_color,eye_color_representation,residents
0,Aleen Minor,unknown,"arid,rocky,windy",Aleena,1,1,1.000000,Ratts Tyerel
1,Cato Neimoidia,10000000,superheated,Neimodian,1,2,0.500000,Nute Gunray
2,Cerea,450000000,unknown,Cerean,1,1,1.000000,Ki-Adi-Mundi
3,Champala,3500000000,"arid,temperate,tropical",Chagrian,1,1,1.000000,Mas Amedda
4,Coruscant,1000000000000,"temperate,frigid",Human,1,6,0.166667,"Finis Valorum,Adi Gallia"
5,Dorin,unknown,temperate,Kel Dor,1,2,0.500000,Plo Koon
6,Endor,30000000,arid,Ewok,1,2,0.500000,Wicket Systri Warrick
7,Glee Anselm,500000000,unknown,Nautolan,1,1,1.000000,Kit Fisto
8,Iktotch,unknown,temperate,Iktotchi,1,1,1.000000,Saesee Tiin
9,Iridonia,unknown,unknown,Zabrak,1,2,0.500000,Eeth Koth


In [805]:
# GROUP BY for average height and mass of species

query = """
SELECT
    s.name AS species_name,
    AVG(p.height) AS average_height,
    AVG(p.mass) AS average_mass
FROM
    people_to_species AS pts
JOIN
    people AS p
    ON pts.id = p.id
JOIN
    species AS s
    ON s.id = pts.species_id
GROUP BY
    s.name
ORDER BY
    average_height DESC;

"""

pd.read_sql_query(sql=query, con=con)

,species_name,average_height,average_mass
0,Quermian,264.000000,0.000000
1,Wookie,231.000000,124.000000
2,Kaminoan,221.000000,44.000000
3,Kaleesh,216.000000,159.000000
4,Gungan,208.666667,49.333333
5,Pau'an,206.000000,80.000000
6,Cerean,198.000000,82.000000
7,Besalisk,198.000000,102.000000
8,Nautolan,196.000000,87.000000
9,Chagrian,196.000000,0.000000


In [795]:
# HAVING to filter directors with more than 1 film about Luke who startered filming before 2000 

query = """ 
SELECT
    f.director AS director,
    MIN(f.release_date) AS first_film_release,
    MAX(f.release_date) AS last_film_release
FROM
    films AS f
INNER JOIN 
    people_to_films as ptf
    ON ptf.films_id = f.id
INNER JOIN
    people as p 
    ON p.id = ptf.id
GROUP BY
    f.director
HAVING
    COUNT(title) > 1
    AND first_film_release < '2000-01-01'
    AND p.name LIKE  '%Luke%'
ORDER BY
    f.director;

"""

pd.read_sql_query(sql=query, con=con)


,director,first_film_release,last_film_release
0,George Lucas,1977-05-25 00:00:00,2005-05-19 00:00:00
1,Irvin Kershner,1980-05-17 00:00:00,1980-05-17 00:00:00
2,Richard Marquand,1983-05-25 00:00:00,1983-05-25 00:00:00


In [798]:
# Common Table Expression (CTE) for various statistics for directors

query = """
WITH DirectorStatistics AS (
    SELECT
        director,
        COUNT(title) AS film_count,
        AVG(episode_id) AS average_episode_number,
        MAX(release_date) AS max_release_date
    FROM
        films
    GROUP BY
        director
)
SELECT
    ds.director,
    ds.film_count,
    ds.average_episode_number,
    ds.max_release_date,
    CASE
        WHEN ds.average_episode_number > 3 THEN 'High Episode'
        ELSE 'Low Episode'
    END AS episode_category
FROM
    DirectorStatistics AS ds
ORDER BY
    ds.director;

"""

pd.read_sql_query(sql=query, con=con)


,director,film_count,average_episode_number,max_release_date,episode_category
0,George Lucas,4,2.5,2005-05-19 00:00:00,Low Episode
1,Irvin Kershner,1,5.0,1980-05-17 00:00:00,High Episode
2,Richard Marquand,1,6.0,1983-05-25 00:00:00,High Episode
